The project aims to take input as paragraphs/text and extract problem statements from junk.
The logic followed is that problem statements , though scattered , together create one meaningful text slab.
We beleive the problem statement to have negative inclination and hence we use sentiment analysis to segregate the negative 
groups.

We use ALLENNLP TEXT ENTAILMENT to group together the problem Statements.
We then treat group elements 

We then Use ALLENNLP Sentiment Analysis to label the groups as postive or negative.

In [1]:
import pandas as pd
from allennlp.predictors.predictor import Predictor
import re
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import wait, ALL_COMPLETED

In [2]:
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/decomposable-attention-elmo-2018.02.19.tar.gz")
predictor_s = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/sst-2-basic-classifier-glove-2019.06.27.tar.gz")


_jsonnet not loaded, treating C:\Users\lsethia\AppData\Local\Temp\tmpbqfz7n9n\config.json as json
_jsonnet not loaded, treating snippet as json
Did not use initialization regex that was passed: .*token_embedder_tokens\._projection.*weight
_jsonnet not loaded, treating C:\Users\lsethia\AppData\Local\Temp\tmpd44ilup4\config.json as json


In [3]:
data=pd.read_excel(r"train.xlsx")

In [55]:
data.shape

(5367, 15)

In [56]:
def clean_text(text):
    text=str(text).lower().split("\n") 
    clean=[]
    for i in text:
        temp="".join(a for a in i if a.isalnum() or a==" ")
#         temp=re.sub(r'\w*\d\w*', '',temp).strip()
        clean.append(temp)
    clean=set(clean)
    cleant=list(clean)
    clean_text="\n".join(i for i in clean)
    return clean_text
    

In [57]:
X=data['Description'].apply(clean_text)

length is  1
['check for oem blackout']


We create a mapping for the data so that we can insert the data into the correct postion during parallel processing 

In [58]:
mapped=[]
for i in range(len(X)):
    mapped.append([i,X[i]])    

##### Here n denotes the number of entires to be parsed into groups 

In [ ]:
n=300
groups=[0]*n
def entail(inp):
    i=inp[1]
    i=re.sub(" +"," ",i)
    reso_g=[]
    temp=i.split("\n")
    temp=[a for a in temp if len(a)>1]
    length=len(temp)
    print("length is ",length)
    print(temp)
    if(length==1):
        groups[inp[0]]=temp
    else:       
        for j in range(len(temp)):
            if(temp[j]!="-1"):                
                group_temp=[temp[j]]
                for k in range(j+1,len(temp)):
                    if(temp[k]!="-1"):
                        score=predictor.predict(hypothesis=temp[j],premise=temp[k])
                        if(score["label_probs"][0]>0.65):
                            group_temp.append(temp[k])
                            temp[k]="-1"
                reso_g.append(group_temp)               
            else:
                continue
#         groups.append([inp[0],reso_g])
        groups[inp[0]]=reso_g
    return "Done"
            
                
#         while(len(temp)>0):
#             k=0
#             while(k<len(temp)):
                
#             for k in range(j,len(temp)):            
#             score=predictor.predict(hypothesis=temp[j],
#               premise=temp[k]
#             )
#             if(score["label_probs"][0]>0.65):
with ThreadPoolExecutor(max_workers=3) as executor:
    future = executor.map(entail, mapped[:n])
    executor.shutdown(wait=True)
    print("done")

Join group data into a single sentence 

In [44]:
join_group=[]
for i in groups:
    gro=[]
    if(type(i) is not int):
        for j in i:
            temp=" ".join(f for f in j )
            gro.append(temp)
        join_group.append(gro)
    else:
        join_group.append([str(i)])

In [67]:
len(groups)

307

Generate sentiment scores

In [91]:
sentiment=[]
for i in join_group:
    temp=[]
    for j in i:
        
        f=predictor_s.predict(
  sentence=j
)
        if(f['label']=='0'):
            temp.append(f)
        else:
            temp.append(f)
    sentiment.append(temp)

In [92]:
len(sentiment)

307

In [93]:
sentiment[0]

[{'logits': [-3.6275806427001953, 3.6046507358551025],
  'probs': [0.0007223839638754725, 0.999277651309967],
  'label': '0'},
 {'logits': [-2.507786750793457, 2.4919331073760986],
  'probs': [0.006694715470075607, 0.9933052659034729],
  'label': '0'},
 {'logits': [1.5602668523788452, -1.4158766269683838],
  'probs': [0.9514846205711365, 0.04851534590125084],
  'label': '1'},
 {'logits': [1.6364237070083618, -1.562311053276062],
  'probs': [0.9607866406440735, 0.03921336308121681],
  'label': '1'},
 {'logits': [-3.457111358642578, 3.5013718605041504],
  'probs': [0.0009496345301158726, 0.9990503191947937],
  'label': '0'},
 {'logits': [-1.28526771068573, 1.3379082679748535],
  'probs': [0.06766165792942047, 0.9323383569717407],
  'label': '0'},
 {'logits': [-2.947356700897217, 3.0252346992492676],
  'probs': [0.0025411571841686964, 0.9974588751792908],
  'label': '0'},
 {'logits': [3.376458168029785, -3.2564785480499268],
  'probs': [0.9986854195594788, 0.001314561697654426],
  'label'

In [54]:
len(join_group)

307

In [53]:
len(sentiment)

307

In [ ]:
inp={
    'description':mapped[:n],
    'groups':groups,
    'sentiments':sentiment
}
inp=pd.DataFrame(inp)
inp.to_excel("TExt entailment + Sentiment analysis.xlsx")